In [1]:
import numpy as np
import em_algorithm
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models


## Experiment 1: MIRT-2PL Parameter Recovery

In [2]:
from unittest import result


def mirt_param_recovery(sample_size) -> dict:
    latent_dimension = 2
    item_dimension = 5

    #Define Population
    cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[1,0],
                [1,0],
                [0,1],
                [1,1],
                [1,1]])
    delta = np.array([0, 0.5, 1, 0, 0.5])
    early_item_parameters = {"A": A, "delta": delta, "item_dimension": item_dimension, "latent_dimension": latent_dimension}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters

    result_dict = {"sample": sample,
                   "early_item_parameters": early_item_parameters,
                   "latent_covariance": cov,
                   "estimated_parameters": None}
    return(result_dict)


In [5]:
U = mirt_param_recovery(100)